*Подключаем необходимые библиотеки*

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import torchvision.datasets as datasets
from torchvision import transforms
from torchvision.models import resnet34, ResNet34_Weights
from torchvision.transforms.functional import InterpolationMode

from tqdm import tqdm

import wandb

*Аунтификация в wandb для отслеживания метрик*

In [2]:
with open("wandb_api.txt", 'r') as file:
    login = file.read()
    wandb.login(key=login)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: da-shumilin03. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


*Инициализация устройства, на котором будут выполняться вычисления*

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

*Преобразования для изображений*

In [4]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2470, 0.2435, 0.2616))
    ]
)

*Загрузка датасетов CIFAR-10*

In [5]:
train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

train_dataloader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

100%|██████████| 170M/170M [00:04<00:00, 39.5MB/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


*Написание модели, использующей свёрточные нейронные сети*

In [6]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv_1 = nn.Conv2d(3, 128, 3, dilation=2)
        self.conv_2 = nn.Conv2d(128, 256, 3, stride=2)
        self.conv_3 = nn.Conv2d(256, 512, 3, dilation=2)

        self.fc_1 = nn.Linear(512, 256)
        self.fc_2 = nn.Linear(256, 128)
        self.fc_3 = nn.Linear(128, 10)

        self.lrelu = nn.LeakyReLU()
        
        self.gap = nn.AdaptiveAvgPool2d(1)

        self.bn2d_1 = nn.BatchNorm2d(128)
        self.bn2d_2 = nn.BatchNorm2d(256)
        self.bn2d_3 = nn.BatchNorm2d(512)
        self.drop = nn.Dropout(0.5)

    def forward(self, x):
        x = self.lrelu(self.bn2d_1(self.conv_1(x)))
        x = self.lrelu(self.bn2d_2(self.conv_2(x)))
        x = self.lrelu(self.bn2d_3(self.conv_3(x)))
        x = self.gap(x)
        x = x.squeeze(-1).squeeze(-1)
        x = self.drop(self.lrelu(self.fc_1(x)))
        x = self.drop(self.lrelu(self.fc_2(x)))
        x = self.fc_3(x)
        
        return x

*Инициализация проекта в wandb*

In [7]:
wandb.init(project="CIFAR-10", name="my_model_cnn")

*Создание модели, добавление отслеживания модели, создание алгоритма оптимизации модели, создание алгоритма изменения шага обучения*

In [8]:
model = MyModel().to(device)
wandb.watch(model)
optimizer = torch.optim.AdamW(model.parameters())
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=15, eta_min=1e-3)

*Обучение модели и отслеживание метрик*

In [9]:
epochs = 15
for epoch in tqdm(range(epochs)):
    model.train()
    for x_train, y_train in train_dataloader:
        x_train, y_train = x_train.to(device), y_train.to(device)
        y_pred = model(x_train)
        loss = F.cross_entropy(y_pred, y_train)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    scheduler.step()

    loss_array_for_iter = np.zeros(len(test_dataloader))
    accuracy_array_for_iter = np.zeros(len(test_dataloader))
    model.eval()
    with torch.no_grad():
        for index, value in enumerate(test_dataloader):
            x_test, y_test = value[0].to(device), value[1].to(device)
            y_pred = model(x_test)
            loss = F.cross_entropy(y_pred, y_test)

            loss_array_for_iter[index] = loss.item()
            accuracy_array_for_iter[index] = (torch.argmax(y_pred, dim=-1) == y_test).cpu().numpy().mean()

        wandb.log(
            {
                "mean loss test": loss_array_for_iter.mean(),
                "mean accuracy test": accuracy_array_for_iter.mean()
            }
        )
        
wandb.finish()

100%|██████████| 15/15 [04:37<00:00, 18.51s/it]


mean accuracy test,▁▃▃▄▅▆▆▇▇▇███▇█
mean loss test,█▆▅▅▄▃▂▂▁▂▁▁▂▂▁
mean accuracy test,0.80932
mean loss test,0.62427


In [11]:
transform = transforms.Compose(
    [
        transforms.Resize(256, interpolation=InterpolationMode.BILINEAR),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

In [12]:
train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

train_dataloader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [13]:
model = resnet34()
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

*Использование предобученной модели, которая будет оптимизировать веса для все модели или только для последнего слоя (голова - классификатор)*

In [14]:
class MyResNet34(nn.Module):
    def __init__(self, full_optimize):
        super().__init__()
        
        self.full_optimize = full_optimize

        self.model = resnet34(weights=ResNet34_Weights.IMAGENET1K_V1)

        if (self.full_optimize):
            self.model.fc = nn.Linear(512, 10)
            self.optimizer = torch.optim.AdamW(self.model.parameters())
        else:
            self.model.fc = nn.Identity()
            self.classifier = nn.Linear(512, 10)
            self.optimizer = torch.optim.AdamW(self.classifier.parameters())

    def forward(self, x):
        if (self.full_optimize):
            x = self.model(x)
        else:
            with torch.no_grad():
                features = self.model(x)
            x = self.classifier(features)

        return x

In [15]:
wandb.init(project="CIFAR-10", name="pretrained_model_cnn")

In [16]:
model = MyResNet34(full_optimize=1).to(device)
wandb.watch(model)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 180MB/s]


In [17]:
epochs = 10
for epoch in tqdm(range(epochs)):
    for x_train, y_train in train_dataloader:
        x_train, y_train = x_train.to(device), y_train.to(device)
        y_pred = model(x_train)
        loss = F.cross_entropy(y_pred, y_train)
        loss.backward()
        model.optimizer.step()
        model.optimizer.zero_grad()

    loss_array_for_iter = np.zeros(len(test_dataloader))
    accuracy_array_for_iter = np.zeros(len(test_dataloader))
    with torch.no_grad():
        for index, value in enumerate(test_dataloader):
            x_test, y_test = value[0].to(device), value[1].to(device)
            y_pred = model(x_test)
            loss = F.cross_entropy(y_pred, y_test)

            loss_array_for_iter[index] = loss.item()
            accuracy_array_for_iter[index] = (torch.argmax(y_pred, dim=-1) == y_test).cpu().numpy().mean()

        wandb.log(
            {
                "mean loss test": loss_array_for_iter.mean(),
                "mean accuracy test": accuracy_array_for_iter.mean()
            }
        )

wandb.finish()

100%|██████████| 10/10 [26:55<00:00, 161.56s/it]


mean accuracy test,▁▅▅▇▇▇▇███
mean loss test,█▃▄▂▁▂▃▂▂▂
mean accuracy test,0.89321
mean loss test,0.40692


*Если судить по средней доле правильных ответов, то мы видим ожидаемый результат, что модель, обученная на большой выборке, у которой была включены оптимизация всех весов выигрывает модель, написанную с нуля*